In [58]:
file = open("/Users/danjane/cltk_data/latin/text/latin_text_latin_library/falcone.txt", "r") 
text = file.read()

# Replace j/v and tokenize
from cltk.stem.latin.j_v import JVReplacer
from cltk.tokenize.word import WordTokenizer

jv_replacer = JVReplacer()
text = jv_replacer.replace(text.lower())

word_tokenizer = WordTokenizer('latin')
text_word_tokens = word_tokenizer.tokenize(text)
text_word_tokens = [token for token in text_word_tokens if token not in ['.', ',', ':', ';','*']]

# Keep words more than 3 characters long
text_word_tokens = [token for token in text_word_tokens if len(token) > 3]

# Quick check, printing the first 10 words
print(text_word_tokens[:9])

['chronicon', 'beneuentanum', 'falcone', 'beneuento', 'chronicon', 'beneuentanum', '1102.1.1', 'apud', 'urbem']


In [59]:
import re

file = open("LatinGeographicalNames.txt")
places = file.read().lower()
lines = places.split('\n')
latinname_modernname = [line.split('\t') for line in lines]

latin_names = []
modern_names = []
for pair in latinname_modernname:
    if len(pair) > 1:
        latins = pair[0].split(',')
        moderns = pair[1].split(',')
        for latin in latins:
            for modern in moderns:
                latin_names.append(latin.lstrip())
                modern_names.append(modern.lstrip())


#places = re.findall(r"[\w']+", places)
for i in range(10):
        print(latin_names[i] + ', ' + modern_names[i])

canonical latin name (source(s): variant(s)), english name (native language(s)) - older name(s)
canonical latin name (source(s): variant(s)), (other language(s))
canonical latin name (source(s): variant(s)), location(s)
brigantium, bregenz
brigantia, bregenz
carnuntum, bad deutsch-altenburg / petronell
chremisa, krems
idunum, judenburg
iuenna, jaunstein - (podjuna)
iuenna, völkermarkt


In [60]:
places_in_text = list(set(text_word_tokens) & set(latin_names))
print(places_in_text[:5])

['luna', 'solium', 'regalis', 'curia']


In [69]:
import os

#files = [file in os.listdir(
#    "/Users/danjane/cltk_data/latin/text/latin_text_latin_library/")]
files = os.listdir("/Users/danjane/cltk_data/latin/text/latin_text_latin_library/")
files = [file in files if (file.endswith('.txt')) ]
print(files)

SyntaxError: invalid syntax (<ipython-input-69-b572a1f96c71>, line 6)